# LBM 1D Code Example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Problem Definiton

Define diffusivity $D$, domain length $L$ and maximum time $t_{max}$

In [16]:
D = 0.2
L = 1.0
t_max = 1

In [3]:
def ic(x, L):
    return np.exp(-((x - L / 2) / 0.1) ** 2)

## Numerical parameters
Define number of lattice nodes $nx$, number of time steps $nt$ and the weighting factor $\theta$. The refinement factor $rf<1$ allows to increase resolution according to the diffusive scaling.

In [4]:
nx = 16
nt = 80
theta = 1/3
rf = 1/2

## Preprocessing
 - Compute the discretization to be used in the analysis: $nx\leftarrow nx/rf$ and $nt\leftarrow nt/rf^2$
 - Determine lattice spacing $\Delta x = L/nx$ and time step size $\Delta t = t_{max}/nt$
 - Setup an array of lattice node locations (assuming $\Delta x/2$ padding at the left and right side of the $(0,L)$ domain
 - Calculate the relaxation rate $\omega$

In [5]:
nx = int(nx/rf);
nt = int(nt/rf**2);
dx = L/nx;
dt = t_max/nt;
omega = 1/(D*dt/dx**2/theta+0.5);

## Equilibrium
Define the local equilibrium $f^{eq}(u)$

In [6]:
def feq(u):
    return np.array([
        (1 - theta) * u,
        (theta / 2) * u,
        (theta / 2) * u
    ])

## Definition of a single LBM time step
 - Compute zeroth-order moment $u^{num} = \sum_i f_i$
 - Perform collision
 - Perform streaming
 - Apply periodic boundary conditions

In [7]:
def timestep(f, omega):
    unum = np.sum(f, axis=0)
    fpost = omega*feq(unum) + (1-omega)*f

    f[0,:] = fpost[0,:]
    f[1,1:] = fpost[1,:-1]
    f[2,:-1] = fpost[2,1:]

    f[1,0] = fpost[1,-1]
    f[2,-1] = fpost[2,0]
    
    return f, unum

## Time step loop

Define and execute lattice Boltzmann method with given initial condition for $nt$ time steps. The lattice nodes are located at a $\Delta x/2$ offset from the domain boundary, i.e. $x_{domain} = \{\Delta x/2, 3\Delta x/2, \dots, L-\Delta x/2\}$

In [8]:
def lbm(L,dx,nx,nt,omega):

    u_out = np.zeros((nt,nx))
    x_domain = np.arange(dx/2, L, dx)
    
    f = feq(ic(x_domain,L))
    for it in range(nt):
        f, u_out[it,:] = timestep(f, omega)
    return x_domain, u_out

In [9]:
x_domain, u_out = lbm(L,dx,nx,nt,omega)

## Visualization

Create animation of $u^{num}$

In [14]:
def animate_u_interval(u_out, x_domain, interval=1):
    """
    Animate u_out over x_domain, showing the first and last frame,
    and intermediate frames at a specified interval.

    Parameters:
        u_out (ndarray): shape (nt, nx), rows = time steps
        x_domain (ndarray): spatial coordinates
        interval (int): step between frames (skip this many frames)
    """
    nt = u_out.shape[0]
    
    # Build frame list: always include first and last, skip intermediate frames
    if nt > 2:
        frames = [0] + list(range(interval, nt-1, interval)) + [nt-1]
    else:
        frames = list(range(nt))  # if very few frames, include all
    
    fig, ax = plt.subplots(figsize=(6, 4))
    line, = ax.plot([], [], lw=2)
    ax.set_xlim(x_domain[0], x_domain[-1])
    ax.set_ylim(1.1*np.min(u_out), 1.1*np.max(u_out))
    ax.set_xlabel("x")
    ax.set_ylabel("u")
    ax.grid(True)
    
    def init():
        line.set_data([], [])
        return line,
    
    def update(frame_idx):
        frame = frames[frame_idx]
        line.set_data(x_domain, u_out[frame, :])
        ax.set_title(f"Iteration = {frame}")
        return line,
    
    ani = FuncAnimation(fig, update, frames=len(frames), init_func=init,
                        blit=True, interval=100)
    
    plt.close(fig)  # prevent static plot from showing
    return HTML(ani.to_jshtml())

In [15]:
animate_u_interval(u_out, x_domain, 10)